In [15]:
from pathlib import Path
import topagnps2cche1d.tools as t2c

In [28]:
import importlib
importlib.reload(t2c);

In [12]:
path_to_og_agflow_file = Path('../input_data/goodwin_creek_1m/AgFlow_Reach_Data.csv')
path_to_asc_file = Path('../input_data/goodwin_creek_1m/AnnAGNPS_Reach_IDs.asc')

In [5]:
df_agflow = t2c.read_agflow_reach_data(path_to_og_agflow_file)

In [6]:
df_agflow

,Reach_ID,Upstream_End_Row,Upstream_End_Column,Downstream_End_Row,Downstream_End_Column,Receiving_Reach,Drainage_Area_[ha],Average_Elevation_[m],Reach_Length_[m],Distance_Upstream_End_to_Outlet_[m],Distance_Downstream_End_to_Outlet_[m],Reach_Slope_[m/m],Contributing_Cell_ID_Source,Contributing_Cell_ID_Right,Contributing_Cell_ID_Left
0,1,6048,578,6048,578,1,2135.57,67.45,0.00,0.00,0.00,0.00283,0,0,0
1,2,5978,761,6048,578,1,2135.57,67.45,247.14,247.14,0.00,0.00283,0,22,23
2,3,5947,822,5978,762,2,2128.12,67.82,75.01,322.15,247.14,0.00134,0,32,33
3,4,5861,859,5946,823,3,2103.89,67.94,103.91,426.06,322.15,0.00097,0,42,43
4,5,5556,1051,5860,859,4,2101.50,68.30,406.73,832.79,426.06,0.00172,0,52,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,908,5980,1251,5973,1174,907,2.91,77.17,119.44,1118.95,999.51,0.02845,9081,9082,9083
907,909,6017,1201,5974,1174,907,1.48,75.50,55.77,1055.28,999.51,0.00002,9091,9092,9093
908,910,5977,892,5979,762,2,6.19,71.00,159.58,406.72,247.14,0.02445,0,9102,9103
909,911,5949,951,5976,893,910,2.57,71.81,70.60,477.32,406.72,0.00281,9111,9112,9113


In [30]:
_, geoMatrix, _, _, _, _ = t2c.read_esri_asc_file(path_to_asc_file, read_array=False)

In [37]:
from topagnps2cche1d.tools import rowcol2latlon_esri_asc

In [40]:
df_agflow = t2c.augment_df_agflow_with_XY(df_agflow, geoMatrix)

In [41]:
df_agflow

,Reach_ID,Upstream_End_Row,Upstream_End_Column,Downstream_End_Row,Downstream_End_Column,Receiving_Reach,Drainage_Area_[ha],Average_Elevation_[m],Reach_Length_[m],Distance_Upstream_End_to_Outlet_[m],Distance_Downstream_End_to_Outlet_[m],Reach_Slope_[m/m],Contributing_Cell_ID_Source,Contributing_Cell_ID_Right,Contributing_Cell_ID_Left,x_US,y_US,x_DS,y_DS
0,1,6048,578,6048,578,1,2135.57,67.45,0.00,0.00,0.00,0.00283,0,0,0,3.791752e+06,231578.114013,3.791752e+06,231578.114013
1,2,5978,761,6048,578,1,2135.57,67.45,247.14,247.14,0.00,0.00283,0,22,23,3.791822e+06,231761.114013,3.791752e+06,231578.114013
2,3,5947,822,5978,762,2,2128.12,67.82,75.01,322.15,247.14,0.00134,0,32,33,3.791853e+06,231822.114013,3.791822e+06,231762.114013
3,4,5861,859,5946,823,3,2103.89,67.94,103.91,426.06,322.15,0.00097,0,42,43,3.791939e+06,231859.114013,3.791854e+06,231823.114013
4,5,5556,1051,5860,859,4,2101.50,68.30,406.73,832.79,426.06,0.00172,0,52,53,3.792244e+06,232051.114013,3.791940e+06,231859.114013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,908,5980,1251,5973,1174,907,2.91,77.17,119.44,1118.95,999.51,0.02845,9081,9082,9083,3.791820e+06,232251.114013,3.791827e+06,232174.114013
907,909,6017,1201,5974,1174,907,1.48,75.50,55.77,1055.28,999.51,0.00002,9091,9092,9093,3.791783e+06,232201.114013,3.791826e+06,232174.114013
908,910,5977,892,5979,762,2,6.19,71.00,159.58,406.72,247.14,0.02445,0,9102,9103,3.791823e+06,231892.114013,3.791821e+06,231762.114013
909,911,5949,951,5976,893,910,2.57,71.81,70.60,477.32,406.72,0.00281,9111,9112,9113,3.791851e+06,231951.114013,3.791824e+06,231893.114013
